# Random Background Analysis

In [4]:
### header ###
__author__ = "Jenhan Tao"
__license__ = "BSD"
__email__ = "jenhantao@gmail.com"

### imports ###
import sys
import os
import pandas as pd
import numpy as np
import argparse
import matplotlib
import itertools
import scipy
import matplotlib.pyplot as plt 
import seaborn as sns
import scipy
import pickle
from sklearn import preprocessing
import sklearn
from sklearn import decomposition
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import KMeans
from sklearn import svm, datasets
from sklearn.utils import shuffle
from sklearn.metrics import roc_curve, auc
from sklearn import ensemble
from sklearn import neighbors
import matplotlib_venn
from sklearn.cross_validation import train_test_split
from random import shuffle

### notebook specific configuration ###
%matplotlib inline
matplotlib.pylab.rcParams['savefig.dpi'] = 200
sys.setrecursionlimit(5000)
os.chdir('/gpfs/data01/glasslab/home/jtao/analysis/random_background_analysis/')
sns.set_context('notebook')
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Functions

In [12]:
# split data into GC content matched training and test data
def get_GC_matched_split(features, labels, test_size, tolerance = 0.01):
    '''
    feature: 2D array (samples x features)
    labels: 1D boolean array (samples x)
    test_size: fraction of data to test on
    tolerance: max difference in GC content between True and False labelled samples
    '''
    global _id_sequence_dict
    
    ### match GC content of samples labelled True with those labelled False by thowing out False samples
    # retrieve sequences using index of labels
    index_label_tuples = tuple(zip(labels.index.values, labels.values))
    
    true_sequences = [_id_sequence_dict[x[0]] for x in index_label_tuples if x[1]]
    true_ids = [x[0] for x in index_label_tuples if x[1]]
    
    false_sequences = [_id_sequence_dict[x[0]] for x in index_label_tuples if not x[1]]
    false_ids = [x[0] for x in index_label_tuples if not x[1]]
    
    # calculate GC content of True samples
    true_gc_count = 0
    true_length = 0
    for s in true_sequences:
        true_gc_count += s.count('G')
        true_gc_count += s.count('C')
        true_length += len(s)
    true_gc_content = true_gc_count/(true_length+0.0000001)
    
    # calcuate GC content of False samples
    false_gc_count = 0
    false_length = 0
    for s in false_sequences:
        false_gc_count += s.count('G')
        false_gc_count += s.count('C')
        false_length += len(s)
    false_gc_content = false_gc_count/(false_length+0.0000001)
    
    while abs(true_gc_content - false_gc_content) > tolerance:
        # remove false GC sequences until GC content matches tolerance
        selected_seq = False
        
        while not selected_seq:
            rand_index = np.random.randint(len(false_sequences))
            current_seq = false_sequences[rand_index]
            current_gc_count = current_seq.count('G')+ current_seq.count('C')
            current_length = len(current_seq)
            current_gc = current_gc_count/current_length
            if true_gc_content > false_gc_content:
                # remove sequences that would increase overall GC content of False sequences
                if current_gc < false_gc_content:
                    selected_seq = True
            else:
                # remove sequences that would decrease overall GC content of False sequences
                if current_gc > false_gc_content:
                    selected_seq = True
        false_gc_count -= current_gc_count
        false_length -= current_length
        false_gc_content = false_gc_count/false_length
        
        false_sequences.pop(rand_index)
        false_ids.pop(rand_index)
    
    filtered_ids = true_ids + false_ids
    filtered_features = features[features.index.isin(filtered_ids)]
    filtered_labels = labels[labels.index.isin(filtered_ids)]

    if test_size <= 0.5:
        training_indices, test_indices = next(iter(
                sklearn.cross_validation.StratifiedKFold(filtered_labels, int(1/test_size), shuffle=True)))
    else:
        test_indices, training_indices = next(
            iter(sklearn.cross_validation.StratifiedKFold(filtered_labels, int(1/(1-test_size)), shuffle=True)))
    training_ids = [filtered_ids[i] for i in training_indices]
    test_ids = [filtered_ids[i] for i in test_indices]
    
    training_features = filtered_features[filtered_features.index.isin(training_ids)]
    test_features = filtered_features[filtered_features.index.isin(test_ids)]
    training_labels = filtered_labels[filtered_labels.index.isin(training_ids)]
    test_labels = filtered_labels[filtered_labels.index.isin(test_ids)]
    
    return training_features, test_features, training_labels, test_labels
    

## Copy Score Files

In [6]:
%%bash
cp /gpfs/data01/glasslab/home/jtao/analysis/cobinding_motif_analysis/motif_score_frame_C57BL6J.pickle ./
cp /gpfs/data01/glasslab/home/jtao/analysis/cobinding_motif_analysis/motif_sequence_frame_C57BL6J.pickle ./
cp /gpfs/data01/glasslab/home/jtao/analysis/cobinding_motif_analysis/motif_strand_frame_C57BL6J.pickle ./
cp /gpfs/data01/glasslab/home/jtao/analysis/cobinding_motif_analysis/motif_start_frame_C57BL6J.pickle ./
cp /gpfs/data01/glasslab/home/jtao/analysis/cobinding_motif_analysis/motif_end_frame_C57BL6J.pickle ./
cp /gpfs/data01/glasslab/home/jtao/analysis/cobinding_motif_analysis/summary_frame.pickle ./
cp /gpfs/data01/glasslab/home/jtao/analysis/cobinding_motif_analysis/annotation_frame.pickle ./
cp /gpfs/data01/glasslab/home/jtao/analysis/cobinding_motif_analysis/peak_sequences/C57BL6J.fa ./

## Read in Score Files

In [8]:
motif_score_frame=pd.read_pickle('motif_score_frame_C57BL6J.pickle')
motif_sequence_frame = pd.read_pickle('motif_sequence_frame_C57BL6J.pickle')
motif_strand_frame = pd.read_pickle('motif_strand_frame_C57BL6J.pickle')
motif_start_frame = pd.read_pickle('motif_start_frame_C57BL6J.pickle')
motif_end_frame = pd.read_pickle('motif_end_frame_C57BL6J.pickle')
summary_frame = pd.read_pickle('summary_frame.pickle')
annotation_frame = pd.read_pickle('annotation_frame.pickle')

scaler = preprocessing.MinMaxScaler()
normed_motif_frame = pd.DataFrame(scaler.fit_transform(motif_score_frame.ix[:,3:]))
normed_motif_frame.columns = motif_score_frame.columns.values[3:]
normed_motif_frame.index = motif_score_frame.index.values

scaler = preprocessing.StandardScaler()
standardized_motif_frame = pd.DataFrame(scaler.fit_transform(motif_score_frame.ix[:,3:]))
standardized_motif_frame.columns = motif_score_frame.columns.values[3:]
standardized_motif_frame.index = motif_score_frame.index.values

_factors = sorted(list(set([x.split('_')[1] for x in summary_frame.columns if '_' in x])))
_factors.remove('atac')

### read in sequences as dictionary {peakID: sequence}
with open('./C57BL6J.fa') as f:
    data = f.readlines()

_id_sequence_dict = {}
for line in data:
    if line[0] == '>':
        sequenceName = line.strip()[1:]
    else:
        _id_sequence_dict[sequenceName] = line.strip().upper()    

## Run Classifier using Open Chromatin Background

In [10]:
numIterations = 5
ap1_members = ['atf3','cjun', 'fos', 'junb','jund']    
test_size = 0.5
factors = ['atf3','cjun', 'fos', 'junb','jund', 'atac', 'cebpa', 'pu1', 'p65']
# c57bl6_indices = summary_frame[summary_frame['Factors'].str.contains('c57bl6')].index.values  



In [13]:

# for monomers using all motifs
strain = 'c57bl6'
factor_auc_dict = {}
factor_precision_dict = {}
factor_coeff_dict = {}
factor_prob_dict = {}
factor_meanCoeff_dict = {}
factor_intercept_dict = {}
factor_meanIntercept_dict = {}
for treatment in ['veh', 'kla']:
    for monomer in ap1_members:
        c57bl6_indices = summary_frame[summary_frame[['c57bl6_' + x + '_' + treatment for x in factors]].sum(axis=1) > 0].index.values  
        features = standardized_motif_frame[standardized_motif_frame.index.isin(c57bl6_indices)]
        labels = summary_frame[summary_frame.index.isin(c57bl6_indices)][strain + '_' + monomer + '_' + treatment] > 0.0
        if np.sum(labels) >= 100:
            all_aucs = []
            all_coefficients = []
            all_probs = None
            all_precisions = []
            all_intercepts = []
            for i in range(numIterations):  

                # split data into training and test sets
                training_features, test_features, training_labels, test_labels = get_GC_matched_split(
                    features, labels, test_size = test_size, tolerance = 0.01)

                #  Run classifier
                lr_classifier = sklearn.linear_model.LogisticRegression(penalty='l1', n_jobs=-1)

                lr_classifier.fit(training_features, training_labels)
                # retrieve probabilities
                probas_lr = lr_classifier.predict_proba(test_features)

                # score predictions
                current_roc_auc = sklearn.metrics.roc_auc_score(test_labels, probas_lr[:, 1], average = None)
                current_precision = sklearn.metrics.average_precision_score(test_labels, probas_lr[:, 1], average = None)

                all_aucs.append(current_roc_auc)
                all_precisions.append(current_precision)

                # score all sequences
                probs = lr_classifier.predict_proba(features)[:, 1]

                current_coefficients = lr_classifier.coef_.flatten()
                all_coefficients.append(current_coefficients)
                all_intercepts.append(lr_classifier.intercept_[0])
                
                if all_probs == None:
                    all_probs = probs
                else:
                    all_probs = all_probs + probs
            mean_coefficients = np.mean(all_coefficients, axis=0)
            
            factor_auc_dict[monomer + '_' + treatment]= all_aucs
            factor_precision_dict[monomer + '_' + treatment] = all_precisions
            factor_coeff_dict[monomer + '_' + treatment] = all_coefficients
            factor_prob_dict[monomer + '_' + treatment] = all_probs
            factor_meanCoeff_dict[monomer + '_' + treatment] = mean_coefficients
            factor_intercept_dict[monomer + '_' + treatment] = all_intercepts
            factor_meanIntercept_dict[monomer + '_' + treatment] = np.mean(all_intercepts)
            print(monomer + '_' + treatment,
                  'roc:', np.mean(all_aucs), np.var(all_aucs),
                  'precision:', np.mean(all_precisions), np.var(all_precisions),  
                  'numTestPositives:', np.sum(test_labels)
                 )

/gpfs/data01/glasslab/home/jtao/software/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:49: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


atf3_veh roc: 0.836974709714 5.61270208208e-06 precision: 0.711601594732 3.53410618648e-06 numTestPositives: 10991
cjun_veh roc: 0.812956501449 3.90929117893e-06 precision: 0.470326834035 6.15164296518e-06 numTestPositives: 6346
fos_veh roc: 0.85820935437 9.32261322104e-06 precision: 0.354988988947 4.44600984708e-05 numTestPositives: 971
junb_veh roc: 0.685413905353 8.2720549774e-05 precision: 0.0214656931746 2.62037146045e-07 numTestPositives: 244
jund_veh roc: 0.806430962408 3.85320637865e-06 precision: 0.56506012027 2.15782634439e-05 numTestPositives: 9146
atf3_kla roc: 0.831525390212 4.86761024132e-07 precision: 0.802433251593 3.69917466276e-06 numTestPositives: 17245
cjun_kla roc: 0.807262952194 1.08763770612e-06 precision: 0.480549590428 4.3931060878e-06 numTestPositives: 8022
fos_kla roc: 0.832284162804 1.86405547115e-06 precision: 0.661452757926 1.67635553822e-05 numTestPositives: 10670
junb_kla roc: 0.829456310449 2.86007019466e-06 precision: 0.480705564122 1.51660288731e-05 n

## Create background peaks from genomic sequences from each chromosome

In [33]:
def getRandomBackground(target_sequences, 
                        size_ratio = 1.0, 
                        tolerance = 0.01, 
                        N_threshold = 0.5 ):
    '''
    target_sequences: list of target sequences
    size_ratio: number of background sequences relative to target sequences
    tolerance: max difference in GC content between True and False labelled samples
    '''
    # throw away background peaks containing greater than this fraction of N


    ### match GC content of samples labelled True with those labelled False by throwing out False samples
    # retrieve sequences using index of labels
        
    # calculate GC content and length of the target sequences
    target_gc_count = 0
    target_length_count = 0
    for s in target_sequences:
        target_gc_count += s.count('G')
        target_gc_count += s.count('C')
        target_length_count += len(s)
    target_gc_content = target_gc_count/(target_length_count+0.0000001) # GC content of target sequences
    mean_target_length = target_length_count/len(target_sequences) # average length of target sequences
    mean_target_length = int(mean_target_length)
    print(target_gc_content, mean_target_length)
    
    # select random genomic loci 
    
        
#     # calcuate GC content of False samples
#     false_gc_count = 0
#     false_length = 0
#     for s in false_sequences:
#         false_gc_count += s.count('G')
#         false_gc_count += s.count('C')
#         false_length += len(s)
#     false_gc_content = false_gc_count/(false_length+0.0000001)
    
#     while abs(target_gc_content - false_gc_content) > tolerance:
#         # remove false GC sequences until GC content matches tolerance
#         selected_seq = False
        
#         while not selected_seq:
#             rand_index = np.random.randint(len(false_sequences))
#             current_seq = false_sequences[rand_index]
#             current_gc_count = current_seq.count('G')+ current_seq.count('C')
#             current_length = len(current_seq)
#             current_gc = current_gc_count/current_length
#             if target_gc_content > false_gc_content:
#                 # remove sequences that would increase overall GC content of False sequences
#                 if current_gc < false_gc_content:
#                     selected_seq = True
#             else:
#                 # remove sequences that would decrease overall GC content of False sequences
#                 if current_gc > false_gc_content:
#                     selected_seq = True
#         false_gc_count -= current_gc_count
#         false_length -= current_length
#         false_gc_content = false_gc_count/false_length
        
#         false_sequences.pop(rand_index)
#         false_ids.pop(rand_index)
    
#     filtered_ids = target_ids + false_ids
#     filtered_features = features[features.index.isin(filtered_ids)]
#     filtered_labels = labels[labels.index.isin(filtered_ids)]

#     if test_size <= 0.5:
#         training_indices, test_indices = next(iter(
#                 sklearn.cross_validation.StratifiedKFold(filtered_labels, int(1/test_size), shuffle=True)))
#     else:
#         test_indices, training_indices = next(
#             iter(sklearn.cross_validation.StratifiedKFold(filtered_labels, int(1/(1-test_size)), shuffle=True)))
#     training_ids = [filtered_ids[i] for i in training_indices]
#     test_ids = [filtered_ids[i] for i in test_indices]
    
#     training_features = filtered_features[filtered_features.index.isin(training_ids)]
#     test_features = filtered_features[filtered_features.index.isin(test_ids)]
#     training_labels = filtered_labels[filtered_labels.index.isin(training_ids)]
#     test_labels = filtered_labels[filtered_labels.index.isin(test_ids)]
    
#     return training_features, test_features, training_labels, test_labels

In [34]:
getRandomBackground(target_sequences)

0.4856183106287546 375


In [24]:
# for monomers using all motifs using random genomic background
strain = 'c57bl6'
factor_auc_dict = {}
factor_precision_dict = {}
factor_coeff_dict = {}
factor_prob_dict = {}
factor_meanCoeff_dict = {}
factor_intercept_dict = {}
factor_meanIntercept_dict = {}
for treatment in ['veh', 'kla']:
    for monomer in ap1_members:
        c57bl6_indices = summary_frame[summary_frame[['c57bl6_' + x + '_' + treatment for x in factors]].sum(axis=1) > 0].index.values  
        features = standardized_motif_frame[standardized_motif_frame.index.isin(c57bl6_indices)]

        target_indices = summary_frame[summary_frame[strain + '_' + monomer + '_' + treatment] > 0.0].index.values
        target_sequences = [_id_sequence_dict[x] for x in target_indices]
        print(len(target_indices), len(target_sequences))

22025 22025
12670 12670
1988 1988
493 493
18049 18049
34654 34654
16135 16135
21465 21465
14080 14080
29964 29964


In [ ]:
peakSize = 200 # size of artificial background peaks
N_threshold = 0.5 # throw away background peaks containing greater than this fraction of N

tile_peak_file = open('./group/merged_tile_peaks.tsv','w')

tile_peak_file.write('\t'.join(['#ID', 'chr', 'start', 'end', 'strand\n']))

for f in os.listdir('./mm10_genome/'):
    if '.fa' in f and not f in ['chrY.fa', 'chrM.fa'] :
        current_chromosome = f.split('.')[0]
        
        # create seperate peak file containing just this chromosome
        if not os.path.isdir('./group_by_chromosome/'+current_chromosome):
            os.mkdir('./group_by_chromosome/'+current_chromosome)
        current_tile_peak_file = open(
            './group_by_chromosome/'+ current_chromosome + '/' + current_chromosome +'_tile_peaks.tsv','w')
        current_tile_peak_file.write('\t'.join(['#ID', 'chr', 'start', 'end', 'strand\n']))
        
        # read in fastq file line by line
        with open('./mm10_genome/' + f) as chromosome_file:
            data = chromosome_file.readlines()
        genome_seq = ''
        for line in data[1:]:
            genome_seq += line.strip().upper()
        genome_seq_size = len(genome_seq)
        
        # create array indicating if position is N
        is_N = [True if x == 'N' else False for x in genome_seq]

        # create array indicating if position overlaps with a peak
        # simultaneously write ATAC-seq peaks
        is_peak = [False for x in genome_seq]
        current_peak_frame = peak_frame[(peak_frame['chr'] == current_chromosome) &
                                        (peak_frame['Factors'].str.contains('Veh'))] # only work with vehicle peaks for now
        print(current_chromosome, current_peak_frame.shape)
        positions = list(zip(current_peak_frame['ID'].values, current_peak_frame['start'].values, current_peak_frame['end'].values))
        for pos in positions:
            
            ID = pos[0]
            start = pos[1] - 1
            end = pos[2]
            for i in range(start, end):
                is_peak[i] = True
            tile_peak_file.write('\t'.join([ID, current_chromosome, str(start), str(end), '+\n']))  
            current_tile_peak_file.write('\t'.join([ID, current_chromosome, str(start), str(end), '+\n']))  
        
        # write background peak files
        for i in range(0,genome_seq_size, peakSize):
            n_count = np.sum(is_N[i:i+peakSize+1])
            if not n_count > peakSize * N_threshold:
                if not np.sum(is_peak[i:i+peakSize+1]) > 0:
                    tile_peak_file.write('\t'.join(['tile_'+str(i), current_chromosome, str(i), str(i+peakSize), '+\n']))        
                    current_tile_peak_file.write('\t'.join([current_chromosome + '_tile_'+str(i), current_chromosome, str(i), str(i+peakSize), '+\n']))        

        current_tile_peak_file.close()
tile_peak_file.close()  

In [ ]:
peakDirectory =  './group_by_chromosome/'
for chrom in os.listdir(peakDirectory):
    peakPath = peakDirectory + chrom + '/' + chrom + '_tile_peaks.tsv'
    seqPath = peakPath.replace('_peaks.tsv','.fa')
    !homerTools extract $peakPath /bioinformatics/homer/data/genomes/mm10 -fa > $seqPath

    

In [ ]:
# create a script to scan for motifs using FIMO
! if [ ! -d ./fimo_results/ ]; then mkdir ./fimo_results/; fi
! if [ ! -d ./fimo_out/ ]; then mkdir ./fimo_out/; fi


pthresh = 0.01
motif_dir = '/home/jenhan/analysis/cobinding_motif_analysis/fimo_motifs/'
fimo_results_dir = './fimo_results'


peakDirectory =  './group_by_chromosome/'
for chrom in os.listdir(peakDirectory):
    scriptFile = open('scanMotifs_' + chrom + '.sh','w')
    for m in os.listdir(motif_dir):
        fimo_out_dir = './fimo_out/' + chrom + '_' + m.replace('.fimo','')

        if 'fimo' in m:
            outPath = fimo_results_dir + '/' +chrom + '_'+ m.replace('.fimo','') +'.txt'
            scriptFile.write(
                'fimo --text --max-stored-scores 2000000 --output-pthresh ' + 
                str(pthresh)  + ' ' +
                motif_dir + '/' + m + ' ./group_by_chromosome/' + chrom + '/' + chrom + '_tile.fa ' +
                '> ' + outPath + ' & \n')
    scriptFile.close()






In [ ]:
%%bash
chmod a+x ./scanMotifs*.sh
for i in ./scanMotifs*sh; 
    do echo 'sleeping...';
    echo $i;
    $i;
    sleep 5m;
done

